In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import requests
import pandas as pd
import json
import os
import plotly.express as px
from datetime import datetime

s = requests.Session()
payload = {'username_or_email': os.getenv("peloton-username"), 'password': os.getenv("peloton-password")}
s.post('https://api.onepeloton.com/auth/login', json=payload)

In [ ]:
url = 'https://api.onepeloton.com/api/me'
# getting data from source
data = s.get(url).json()

pretty_data = json.dumps(data, indent=4)

print(pretty_data)

In [ ]:
peloton_id = data['id']
print(peloton_id)

In [ ]:
workdout_id = '532b8c0fd61743618b86fe8353e9ed49'

url = 'https://api.onepeloton.com/api/workout/'+ workdout_id +'/performance_graph?every_n=5'
# getting data from source

get_data = s.get(url).json()
seconds_since_pedaling_start = get_data['seconds_since_pedaling_start']
output = get_data['metrics'][0]['values']
cadence = get_data['metrics'][1]['values']
resistence = get_data['metrics'][2]['values']
speed = get_data['metrics'][3]['values']
heart_rate = get_data['metrics'][4]['values']


data = {'seconds_since_pedaling_start': seconds_since_pedaling_start,
        'output': output,
        'cadence': cadence,
        'resistence': resistence,
        'speed': speed,
        'heart_rate': heart_rate
       }
df2 = pd.DataFrame(data)
df2.head()

In [ ]:
# Using plotly.express
import plotly.express as px

# df = px.data.stocks()
fig = px.line(df2, 
              x='seconds_since_pedaling_start', 
              y="heart_rate",
              title='Heart rate')

fig1 = px.scatter(df2, x='seconds_since_pedaling_start', 
                       y="output",
                         title='Workout output',  
                         trendline="ols", 
                         trendline_color_override="red",
                    )
fig.show()
fig1.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
                x=df2['seconds_since_pedaling_start'], 
                y=df2["heart_rate"],
                name="BPM"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=df2['seconds_since_pedaling_start'], 
               y=df2["output"],
               name="Output"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Heart BPM and Output"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Heart BPM</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Output</b>", secondary_y=True)

fig.show()